# drive_bf_qt_scraping

A notebook to describing the code used to drive the scraping.

In [1]:
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  
from lxml import html 

In [2]:
class Render(QWebPage):  
    def __init__(self, url):  
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()  

    def _loadFinished(self, result):  
        self.frame = self.mainFrame()  
        self.app.quit() 
    
    def update_url(self, url):
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()  
        

## First get the hotel data links and text summaries

In [3]:
url = 'http://www.bringfido.com/lodging/city/new_haven_ct_us'  
#This does the magic.Loads everything
r = Render(url)  
#result is a QString.
result = r.frame.toHtml()

In [4]:
#QString should be converted to string before processed by lxml
formatted_result = str(result.toAscii())

In [5]:
#Next build lxml tree from formatted_result
tree = html.fromstring(formatted_result)

In [6]:
#Now using correct Xpath we are fetching URL of archives
archive_links = tree.xpath('//*[@id="results_list"]/div')

hotel_names = []
text_summaries = []
links = []

for lnk in archive_links:
    hotel_names.append(lnk.xpath('div[2]/h1/a/text()')[0])
    text_summaries.append(lnk.text_content())
    links.append(lnk.xpath('div/h1/a/@href')[0])
    # print(lnk.text_content())
    # print('*'*25)


In [7]:
links

['/lodging/70449/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=75.01',
 '/lodging/70451/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=219',
 '/lodging/70452/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=109.65',
 '/lodging/70447/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=55.95',
 '/lodging/106805/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=189']

## Now cycle through each hotel, and scrape the reviews, ratings, etc.

In [8]:
url_base = 'http://www.bringfido.com'  

outer_texts = []
outer_titles = []
outer_authors = []
outer_ratings = []
outer_addresses = []
outer_csczip = []
outer_phones = []

for link in links:
    print('*'*50)
    print('Now on {}'.format(link))
    print('*'*50)
    r.update_url(url_base+link)  
    result = r.frame.toHtml()

    #QString should be converted to string before processed by lxml
    formatted_result = str(result.toAscii())

    tree = html.fromstring(formatted_result) 

    hotel_description = tree.xpath('//*[@class="body"]/text()')

    # scrape the address details section of the page
    details = tree.xpath('//*[@class="address"]/text()')

    # now get just the address:
    address = details[0]
    
    # and just the city, state, country, and zip code:
    csczip = details[1]
    
    # and just the phone number
    phone = details[2]

    outer_addresses.append(address)
    outer_csczip.append(csczip)
    outer_phones.append(phone)
    
    #Now using correct Xpath we are fetching URL of archives
    reviews = tree.xpath('//*[@class="review_container"]')

    texts = []
    titles = []
    authors = []
    ratings = []

    print(reviews)
    print('')
    for rev in reviews:
        titles.append(rev.xpath('div/div[1]/text()')[0])
        authors.append(rev.xpath('div/div[2]/text()')[0])
        texts.append(rev.xpath('div/div[3]/text()')[0])
        ratings.append(rev.xpath('div[2]/img/@src')[0].split('/')[-1][0:1])
        print(rev.xpath('div[2]/img/@src')[0].split('/')[-1][0:1])
    outer_texts.append(texts)
    outer_titles.append(titles)
    outer_authors.append(authors)
    outer_ratings.append(ratings)


**************************************************
Now on /lodging/70449/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=75.01
**************************************************
[<Element div at 0x109e69f70>, <Element div at 0x109e69fc8>]

5
3
**************************************************
Now on /lodging/70451/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=219
**************************************************
[]

**************************************************
Now on /lodging/70452/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=109.65
**************************************************
[<Element div at 0x109e3cf70>, <Element div at 0x10b104418>, <Element div at 0x10b104470>, <Element div at 0x10b1044c8>, <Element div at 0x10b104520>, <Element div at 0x10b104578>]

5
5
5
4
4
2
**************************************************
Now on /lodging/70447/?cid=14745&ar=&dt=&rm=1&ad=1&ch=0&dg=1&rt=55.95
**************************************************
[<Element div at 0x10b1043c0>, <Element div at 0x

In [9]:
for idx, hotel_nm in enumerate(hotel_names):
    print('*'*50)
    print(hotel_nm)
    print(outer_addresses[idx])
    print(outer_csczip[idx])
    print(outer_phones[idx])
    print('Number of reviews: {}'.format(len(outer_texts[idx])))
    print(outer_texts[idx])

**************************************************
La Quinta Inn & Suites New Haven

400 Sargent Dr

New Haven, CT, US 06511-5907

877-411-3436
Number of reviews: 2
['My 75lb dog and I received a lovely welcome from two young gentlemen working at the desk the evening of 8/1/14. Check-in was very easy and even though this hotel is close to the highway and Ikea, there is a big patch of grass/bushes out front for dogs to relieve themselves --I didn\'t see bags so bring your own. Yes, I agree with review that it is an older place but my room with one queen bed was plenty clean and comfortable. When I travel with my dog, the less fancy the better. :). Also, it made it nice that there was a dog friendly restaurant less than 10 minute drive away...see review for "Basta". I definitely feel good about spending just over $100 at this hotel which was just a stop along the way in my travels. Recommend to other dog owners. (P.s. This site says small pets only but that is not up to date. The policy 